#### Paphos Weather

This page shows various metrics for the weather in Paphos, Cyprus.

Weather data is purchased from Openweather Marketplace [here](https://home.openweathermap.org/marketplace/my_orders)

Download this weather data from here https://drive.google.com/file/d/14GgJu7YfZulZU9_xpS9GVcykdb1nEaUV/view?usp=sharing




In [220]:
def calculate_heat_index(temperature, relative_humidity):
    heat_index = (-8.784695 +
                  1.61139411 * temperature +
                  (2.338549 * relative_humidity) +
                  (-0.14611605 * temperature * relative_humidity) +
                  (-0.01230809 * (temperature ** 2)) +
                  (-0.01642482 * (relative_humidity ** 2)) +
                  (0.00221173 * (temperature ** 2) * relative_humidity) +
                  (0.00072546 * temperature * (relative_humidity ** 2)) +
                  (-0.000003582 * (temperature ** 2) * (relative_humidity ** 2)))
    return heat_index

In [221]:

from datetime import date
from datetime import datetime
import time
import pandas as pd

def toDate(epoch_time):
    return datetime.fromtimestamp(epoch_time)  

In [222]:
import pandas as pd

df=pd.read_json('/Users/walkerrowe/Documents/hypatia/paphosWeather2023.json')

In [223]:
df.columns

Index(['dt', 'dt_iso', 'timezone', 'main', 'clouds', 'weather', 'wind', 'lat',
       'city_name', 'lon', 'rain'],
      dtype='object')

In [224]:
df['dt'].map(toDate)

0       2018-07-01 03:00:00
1       2018-07-01 04:00:00
2       2018-07-01 05:00:00
3       2018-07-01 06:00:00
4       2018-07-01 07:00:00
                ...        
43843   2023-07-01 22:00:00
43844   2023-07-01 23:00:00
43845   2023-07-02 00:00:00
43846   2023-07-02 01:00:00
43847   2023-07-02 02:00:00
Name: dt, Length: 43848, dtype: datetime64[ns]

We have to convert the epoch time to ISO Date, which is something we can read.

The map function runs a function over a series.  A series means a dataframe with one column.  When we write df['new column'] = df['some column'].map(somefunction) it adds the new columns to the dataframe.


In [225]:
df['dateTime']=df['dt'].map(lambda l: toDate(l).strftime("%Y-%m-%d %H:%M:%S"))

In [226]:
df['month']=df['dt'].map(lambda l: toDate(l).strftime("%m"))
df['day']=df['dt'].map(lambda l: toDate(l).strftime("%d"))
df['year']=df['dt'].map(lambda l: toDate(l).strftime("%y"))
df['hour']=df['dt'].map(lambda l: toDate(l).strftime("%H"))
df['hour']=pd.to_numeric(df['hour'])

In [227]:
df['temp']=df["main"].map(lambda x: x["temp"] )
df['temp_min']=df["main"].map(lambda x: x["temp_min"] )
df['temp_max']=df["main"].map(lambda x: x["temp_max"] )
df['humidity']=df["main"].map(lambda x: x["humidity"])

In [228]:
df['heatIndex'] = df.apply(lambda row: calculate_heat_index(row['temp'], row['humidity']), axis=1)


In [238]:

summer=df.loc[df['month'].isin(['06','07','08'])]
afternoon=df.loc[(df['hour'] > 13)]

In [239]:
hot=afternoon[['year','month','day', 'hour' ,'temp','temp_min','temp_max','hour', 'humidity', 'heatIndex']]

We want to caculate the average temperature by date usign the mean() function.  But the problem is that the date is an object inside the **main** object.  See below how to fix that.

In [240]:
hot.sort_values(by='temp_max',ascending=False).head(50)

,year,month,day,hour,temp,temp_min,temp_max,hour,humidity,heatIndex
16500,20,05,18,15,33.36,33.13,40.96,15,30,32.450873
16499,20,05,18,14,32.61,32.60,39.85,14,36,32.435539
16525,20,05,19,16,29.61,29.60,39.85,16,55,31.247545
16523,20,05,19,14,30.61,30.60,39.85,14,52,32.355522
27132,21,08,04,15,31.91,29.70,39.29,15,55,35.455029
27107,21,08,03,14,31.66,29.27,39.29,14,54,34.695901
27108,21,08,03,15,31.68,29.44,39.29,15,52,34.248330
19115,20,09,04,14,31.61,31.60,39.29,14,79,42.577469
27131,21,08,04,14,32.06,29.53,39.29,14,54,35.494714
27156,21,08,05,15,31.38,28.72,39.29,15,51,33.475092
